####緣由：
1. 上傳資料至Azure時，需重新賦予每一欄資料型別的資訊(即date type、string type 等)，一般習慣讓spark自動辨認資料型別
2. 根據經驗，spark 自動辨認 data type 的方式常出現資料型別辨認錯誤的情況，常見的錯誤如下：
  * data_month 欄位的 date type 資料被誤認為 string type 或是 timestamp type
  * age 欄位的 integer type 被誤認為 string type
3. 為快速校正辨認錯誤的資料型別而撰寫此筆記本

####此Notebook用途：
* 上傳表格後，建立小工具以**手動選取**正確data type的方式進行校正，並寫入指定的資料庫

####此Notebook使用方法：
* 請clone到自己的Workspace使用，若非必要不須改動程式碼
* 將需要上傳的一份檔案準備好，詳情請見【預先準備】
* 順著每一個step的順序執行各cell
* **有建立Widget小工具的地方會需要手動去填入需要的參數，填寫完才能繼續跑下一個cell**
* 如有任何疑問，請就近尋找單位中會使用python的同仁協助

####預先準備：需先至Azure Databricks首頁上傳檔案
1. 上傳表格檔案，並複製上傳後Databricks所顯示的檔案名稱: /FileStore/tables/ **"table.csv"**
  
@version: 2020-08-30


In [0]:
#import package
from pyspark.sql.types import *
from pyspark.sql.functions import *

#remove all widgets
dbutils.widgets.removeAll()

##Step 1 - 讀入表格

In [0]:
#創建Widget小工具
TFLst = ["true","false"]
FormatLst = ['csv','text','json']

dbutils.widgets.text("FileName","檔案名稱.csv")
dbutils.widgets.dropdown("FileFormat","csv",FormatLst)
dbutils.widgets.dropdown("Header", "true", TFLst)
dbutils.widgets.text("Encoding","utf8")
dbutils.widgets.text("Delimiter",",")

In [0]:
#印出小工具內容物
FILENAME = dbutils.widgets.get("FileName")
FILEFORMAT = dbutils.widgets.get("FileFormat")
HEADER = dbutils.widgets.get("Header")
ENCODING = dbutils.widgets.get("Encoding")
DELIMITER = dbutils.widgets.get("Delimiter")

print('上傳後的檔案名稱:', FILENAME)
print('檔案類型:', FILEFORMAT)
print('讀入column名稱:',HEADER)
print('檔案編碼:',ENCODING)
print('分隔符號:',DELIMITER)
print('====若有輸入錯誤，上方小工具更正後，重跑此cell即可====')


#用上面的小工具讀入file content
df = spark.read.format(FILEFORMAT).options(header = HEADER,#是否具有表頭
                                           inferSchema = "true",#是否需要自動辨識data type
                                           encoding= ENCODING, #指定文檔編碼
                                           delimiter = DELIMITER
                                          ).load("/FileStore/tables/"+FILENAME)

#get File Name as DatasetName for further use (as the default value when creating the widgt for writing dataset name ) 
DatasetName = dbutils.widgets.get("FileName")
display(df)

##Step 2 - 手動校正讀入表格的data type

In [0]:
#get df data and df schema 
DFSchema= df.schema.fields

#create a list "DFScheLst" that contain DF data and DF data type
DFScheLst= []
for c in range(len(DFSchema)):
  DFScheLst.append((DFSchema[c].name,str(DFSchema[c].dataType)))


#remove widgets for data import
dbutils.widgets.removeAll()

#create list for dropdown widget, and sort the list
TypeLst = [
  'ByteType',
  'ShortType',
  'IntegerType',
  'LongType',
  'FloatType',
  'DoubleType',
  #'DecimalType(n,0)',
  'StringType',
  'BinaryType',
  'BooleanType',
  'TimestampType',
  'DateType',
  #'ArrayType',
  #'MapType',
  #'StructType',
  #'StructField'
]
TypeLst.sort()

#create widgets for each column 
for n in range(len(DFSchema)):
  dbutils.widgets.combobox(DFScheLst[n][0],DFScheLst[n][1], TypeLst)
print('====跑過這行後會產生各列的data type 的小工具，可在上面選取或是直接輸入====')

In [0]:
#create a dictionery to link widget string to function name
Dic = {
  'ByteType':ByteType(),
  'ShortType':ShortType(),
  'IntegerType':IntegerType(),
  'LongType':LongType(),
  'FloatType':FloatType(),
  'DoubleType':DoubleType(),
  #'DecimalType':DecimalType(),
  'StringType':StringType(),
  'BinaryType':BinaryType(),
  'BooleanType':BooleanType(),
  'TimestampType':TimestampType(),
  'DateType':DateType(),
  #'ArrayType':ArrayType(),
  #'MapType':MapType(),
  #'StructType':StructType(),
  #'StructField':StructField()
}

#modify schema in a new dataframe
##@column name: DFScheLst[n][0]
##@DF data type: DFScheLst[n][1]
##@new data type: dbutils.widgets.get(DFScheLst[n][0])
##@transform data type name to function: Dic[dbutils.widgets.get(DFScheLst[n][0])]
df_2 = df
print('====預覽轉換過的資料表樣式====')
for n in range(len(DFSchema)):
  df_2 = (df_2.withColumn(DFScheLst[n][0],df[DFScheLst[n][0]].cast(Dic[dbutils.widgets.get(DFScheLst[n][0])]))
         ) 
display(df_2)


In [0]:
#show new data
NewSchema= df_2.schema.fields

#create a list"ScheLst" that contain column name, DF data type, new data type
ScheLst = []
for c in range(len(DFSchema)):
  ScheLst.append((c,DFSchema[c].name,str(DFSchema[c].dataType),str(NewSchema[c].dataType)))

#create a spark dataframe that contais all data type
SchemaDF =spark.createDataFrame(ScheLst,["#","Column Name","Old Data Type","New Data Type"])
print('====比較改動前後的資料格式，重複確認是否無誤====')
SchemaDF.show(10000,False)

##Step 3 - 將表格寫入至資料庫

In [0]:
#check file name
dbutils.widgets.removeAll()

#get DatasetName from cell 2
TableName = DatasetName[:-4]

#list for write mode options
WriteModeLst = ['append','overwrite','ignore','error']

#create widgets for writing the table into database  
dbutils.widgets.text("InputTableName",TableName)
dbutils.widgets.text("InputDBName","自家單位使用的資料庫名稱")
dbutils.widgets.dropdown("WriteMode","overwrite",WriteModeLst)

#get and print the parameters 
INPPUTDB = dbutils.widgets.get("InputDBName")
INPUTTABLE = dbutils.widgets.get("InputTableName")
WRITEMODE = dbutils.widgets.get("WriteMode")
print('寫入資料庫:', INPPUTDB)
print('寫入資料表名稱:', INPUTTABLE)
print('寫入模式:', WRITEMODE)

In [0]:
#write to database
df_2.write.format('delta').saveAsTable(INPPUTDB+'.'+INPUTTABLE,mode = WRITEMODE)

##完成!!

In [0]:
%sql
select * from ...